# Упрощённая реализация Tensor‑Train SVD (TT‑SVD)

Минимальный, понятный пример без дополнительных тестов/оптимизаций.

In [5]:
import numpy as np

In [6]:

def choose_rank(s, eps):
    """Выбирает минимальный ранг k, после которого энергия хвоста ≤ eps.
    Если условие не выполняется, берётся полный ранг."""
    for k in range(1, len(s)):
        if np.sum(s[k:] ** 2) <= eps:
            return k
    return len(s)


In [7]:

def tt_svd(tensor, eps=1e-8):
    """Простая версия TT‑SVD.
    Возвращает список факторов: [G₁, …, G_d], где
      • G₁ → (n₁, r₁)
      • G_i → (r_{i-1}, n_i, r_i) для 1<i<d
      • G_d → (r_{d-1}, n_d)"""
    dims = tensor.shape
    d = len(dims)
    factors = []
    core = tensor.reshape(dims[0], -1)

    for i in range(d - 1):
        U, s, Vh = np.linalg.svd(core, full_matrices=False)
        k = choose_rank(s, eps)
        U_k, s_k, Vh_k = U[:, :k], s[:k], Vh[:k, :]

        if i == 0:
            factors.append(U_k)  # (n₁, r₁)
        else:
            r_prev = factors[-1].shape[-1]
            factors.append(U_k.reshape(r_prev, dims[i], k))  # (r_{i-1}, n_i, r_i)

        core = s_k[:, None] * Vh_k  # (k, rest)
        if i < d - 2:
            core = core.reshape(k * dims[i + 1], -1)
        else:
            factors.append(core.reshape(k, dims[i + 1]))  # финальный фактор (r_{d-1}, n_d)

    return factors


In [8]:

def tt_to_tensor(factors):
    res = factors[0]  # (n₁, r₁)
    for G in factors[1:-1]:
        # res ... r_{i-1}   ×   G r_{i-1} n_i r_i → (…, n_i, r_i)
        res = np.tensordot(res, G, axes=([-1], [0]))
    res = np.tensordot(res, factors[-1], axes=([-1], [0]))  # финальный фактор
    return res


In [9]:

# Пример использования
sizes = (10, 15, 20)
T = np.random.randn(*sizes)

factors = tt_svd(T, eps=1e-10)
T_rec = tt_to_tensor(factors)

print("Относительная ошибка восстановления:", np.linalg.norm(T_rec - T) / np.linalg.norm(T))
for i, f in enumerate(factors):
    print(f"G{i+1} shape: {f.shape}")


Относительная ошибка восстановления: 2.270049412395327e-15
G1 shape: (10, 10)
G2 shape: (10, 15, 20)
G3 shape: (20, 20)
